# Advent of Code 2022!

# Day 1

In [18]:
# Day 1: Calorie Counting

import numpy as np
import pandas as pd

# read in the file
f = open('data/1_1.csv')
fl = f.readlines()
f.close()

fl = [f.strip('\n') for f in fl]

# Make a running total of calories. 
# When we finish an elf, append their running total to our list
# Elves are separated by blank lines, ('' == end of elf)
n_elf = 0
elf_amounts = []
elf_tot = 0
for f in fl:
    if f == '':
        n_elf += 1
        elf_amounts.append(elf_tot)
        elf_tot = 0
    else:
        elf_tot += int(f)
# make sure the last calorie count is added
elf_amounts.append(elf_tot)

In [19]:
np.max(elf_amounts)

68787

In [20]:
# Now find the sum of the top three

# throw the values into a pandas DataFrame
elfs = pd.DataFrame({'calories': elf_amounts})
# sort by value, take the last three, and sum them up
elfs.sort_values('calories').iloc[-3:,].sum()

calories    198041
dtype: int64

# Day 2

In [3]:
import numpy as np
import pandas as pd

df = pd.read_csv('data/2_1.dat', sep=' ', header=None)
df

,0,1
0,C,Z
1,C,Y
2,B,X
3,A,Z
4,C,Z
...,...,...
2495,B,Z
2496,C,Y
2497,A,Z
2498,A,Z


In [5]:
df.iloc[0,1]

'Z'

In [23]:
def score(row):
    if row[1] == 'X':
        val = 1
        if row[0] == 'C':
            win = 6
        elif row[0] == 'A':
            win = 3
        else:
            win = 0
    elif row[1] == 'Y':
        val = 2
        if row[0] == 'A':
            win = 6
        elif row[0] == 'B':
            win = 3
        else:
            win = 0
    if row[1] == 'Z':
        val = 3
        if row[0] == 'B':
            win = 6
        elif row[0] == 'C':
            win = 3
        else:
            win = 0
    return win + val

# given A (rock), B (paper), or C (scissors)
# return the number corresponding to the loss
def score_loss(target):
    if target == 'A':
        return 3
    elif target == 'B':
        return 1
    else:
        return 2

# given A (rock), B (paper), or C (scissors)
# return the number corresponding to the win
def score_win(target):
    if target == 'A':
        return 2
    elif target == 'B':
        return 3
    else:
        return 1

def score2(row):
    if row[1] == 'X':
        win = 0
        val = score_loss(row[0])
    elif row[1] == 'Y':
        win = 3
        if row[0] == 'A':
            val = 1
        elif row[0] == 'B':
            val = 2
        else:
            val = 3
    else:
        win = 6
        val = score_win(row[0])
    return win + val


In [26]:
scores = []
for row_id in range(df.shape[0]):
    scores.append(score(df.iloc[row_id,]))
np.sum(scores)

9241

In [24]:
scores2 = []
for row_id in range(df.shape[0]):
    scores2.append(score2(df.iloc[row_id,]))
np.sum(scores2)

14610

# Day 3

In [63]:
import string

score_dict = dict()
for i, letter in enumerate(string.ascii_letters):
    score_dict[letter] = i+1

# read in data
with open('data/3_1.dat', 'r') as f:
    fl = f.readlines()

# remove newline character from end of each line
fl = [s.strip('\n') for s in fl]

def split(x):
    # split string in half
    n = len(x)
    return x[slice(0, int(n/2))], x[slice(int(n/2), n)]

def get_match(l1, l2):
    # use list intersection to find the item in both
    return list(set(l1) & set(l2))[0]

def score(x):
    return score_dict[x]


In [64]:
scores = 0
for line in fl:
    l1, l2 = split(line)
    scores += score(get_match(l1, l2))
scores

7831

In [67]:
# part 2, find the sum of values for items common across each group of 3 lines
def match3(l1, l2, l3):
    return list(set(l1) & set(l2) & set(l3))[0]

scores = 0
ii = 0
# iterate through the list (3 at a time) and calculate the sum
while ii < len(fl)-2:
    l1 = fl[ii]
    l2 = fl[ii+1]
    l3 = fl[ii+2]
    scores += score(match3(l1, l2, l3))
    ii += 3 # don't forget to iterate by 3
scores

2683

# Day 4

In [97]:
# Day 4: Camp Cleanup

# There are two columns, which represent two ranges of data
# We want to know if one range every completely encompasses
# the other range
#   e.g. 6-6 is consumed by 4-6

# Let's use pandas and it's built in str functionality
import pandas as pd

df = pd.read_csv('data/4_1.dat', header=None)
df

,0,1
0,8-13,10-65
1,13-23,14-24
2,72-83,73-76
3,37-37,39-73
4,64-94,65-65
...,...,...
995,49-51,50-55
996,7-20,20-60
997,19-99,20-98
998,83-97,45-82


In [101]:
# pull out the start and end values, and convert to int
d = pd.DataFrame()
d[['x1', 'x2']] = df[0].str.split('-', expand=True)
d[['y1', 'y2']] = df[1].str.split('-', expand=True)
d = d.astype(int)
d

,x1,x2,y1,y2
0,8,13,10,65
1,13,23,14,24
2,72,83,73,76
3,37,37,39,73
4,64,94,65,65
...,...,...,...,...
995,49,51,50,55
996,7,20,20,60
997,19,99,20,98
998,83,97,45,82


In [118]:
# function to check fill out the ranges and find overlap
# later updated to specify minimum overlap required
def check_overlap(row, min=None):
    """Check if ranges (row[0]-row[1]) and (row[2]-row[3])
    overlap by at least `min` values
    
    If min is None, match full length of shortest range
    """
    a = np.arange(row[0], row[1]+1)
    b = np.arange(row[2], row[3]+1)

    if min is None:
        if a.shape[0] > b.shape[0]:
            min = b.shape[0]
        else:
            min = a.shape[0]

    m = list(set(a) & set(b))
    if len(m) >= min:
        return 1
    else:
        return 0

In [119]:
# iterate over rows to find complete subsets
matches = 0
for rid in range(d.shape[0]):
    matches += check_overlap(d.iloc[rid,])
matches

526

In [121]:
# iterate over rows to find any overlap
matches = 0
for rid in range(d.shape[0]):
    matches += check_overlap(d.iloc[rid,], min=1)
matches

886

# Day 5: Supply Stacks

In [296]:
def get_stacks():
    stacks_orig = [
        ['W', 'T', 'H', 'P', 'J', 'C', 'F'],
        ['H', 'B','J', 'Z', 'F', 'V', 'R', 'G'],
        ['R', 'T', 'P', 'H'],
        ['T', 'H', 'P', 'N', 'S', 'Z'],
        ['D', 'C', 'J', 'H', 'Z', 'F', 'V', 'N'],
        ['Z', 'D', 'W', 'F', 'G', 'M', 'P'],
        ['P', 'D', 'J', 'S', 'W', 'Z', 'V', 'M'],
        ['S', 'D', 'N'],
        ['M', 'F', 'S', 'Z', 'D']
    ]
    for stack in stacks_orig:
        stack.reverse()
    return stacks_orig

In [302]:
def move(n, from_, to_, stacks):
    for ii in range(n):
        crate = stacks[from_].pop()
        stacks[to_].append(crate)
    return stacks

def multi_move(n, from_, to_, stacks2, debug=False):
    crates = []
    for ii in range(n):
        crates.append(stacks2[from_].pop())

    while len(crates) > 0:
        stacks2[to_].append(crates.pop())
    return stacks2



In [266]:
with open('data/5_2.dat', 'r') as f:
    instr = f.readlines()
instructions = [c.strip('\n').split(' ') for c in instr]
instructions[:3]

[['move', '2', 'from', '8', 'to', '2'],
 ['move', '3', 'from', '9', 'to', '2'],
 ['move', '1', 'from', '3', 'to', '8']]

In [287]:
stacks = get_stacks()
for inst in instructions:
    _, n, _, from_, _, to_ = inst
    stacks = move(int(n), int(from_)-1, int(to_)-1, stacks)

In [288]:
for stack in stacks:
    print(stack[-1], end='')

SPFMVDTZT

In [305]:
stacks2 = get_stacks()
for inst in instructions:
    #print(inst)
    _, n, _, from_, _, to_ = inst
    stacks2 = multi_move(int(n), int(from_)-1, int(to_)-1, stacks2)

In [306]:
for stack in stacks2:
    print(stack[-1], end='')

ZFSJBPRFP

# Day 6

In [1]:
with open('data/6_1.dat', 'r') as f:
    stream = f.readlines()

In [18]:
buffer = []

for i, s in enumerate(stream[0]):
    buffer.insert(0, s)
    if len(buffer) > 4:
        buffer.pop()
    if len(set(buffer)) == 4:
        break
i+1

1625

In [21]:
buffer = []

for i, s in enumerate(stream[0]):
    buffer.insert(0, s)
    if len(buffer) > 14:
        buffer.pop()
    if len(set(buffer)) == 14:
        break
i+1

2250